**Importing necessary libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')

**Load the data & Perform a quick EDA (Exploratory Data Analysis)**

In [ ]:
data = pd.read_csv("Apparel_POS.csv", encoding="unicode_escape", parse_dates=['Order Date'])
print("Total number of transactions happened in the given period: "+ str(data.shape[0]))
data.head(5)

Total number of transactions happened in the given period: 32320


,Order No,External Order No,Order Date,Order Type,Status,Customer Name,Country,State,City,Email,...,Primary colour,MRP,product Discounted?,Product Discounted %,Account,New order No.,Customer_ID,State_City,Final state,Final City
0,M011000,1996,2018-01-05 02:08:00,Prepaid,Shipped complete,anindita roy,India,Karnataka,Bengaluru,ro***********@gmail.com,...,Blue,945,YES,30%,A,A1996,14955.0,Karnataka_Bengaluru,Karnataka,Bangalore
1,M0110001,10952,2019-08-22 15:12:00,Prepaid,Shipped complete,Raghu VS,India,Karnataka,Bangalore,az*********@gmail.com,...,Pink,3095,NO,0%,A,A10952,2532.0,Karnataka_Bangalore,Karnataka,Bangalore
2,M0110002,10953,2019-08-22 16:01:00,Prepaid,Shipped complete,sumit s,India,Maharashtra,mumbai,na*****@yahoo.com,...,Black,1095,NO,0%,A,A10953,10850.0,Maharashtra_mumbai,Maharashtra,Mumbai
3,M011001,1997,2018-01-05 05:16:00,Prepaid,Shipped complete,GSRK VARMA,India,Karnataka,Bidar,du***@dummy.com,...,Pink,995,YES,50%,A,A1997,19829.0,Karnataka_Bidar,Karnataka,Bidar
4,M0110013,10964,2019-08-22 22:49:00,Prepaid,Shipped complete,Natasha Sethi,India,Haryana,Gurugram,na*************@gmail.com,...,Multicolor,2995,NO,0%,A,A10964,10993.0,Haryana_Gurugram,Haryana,Gurgaon


In [ ]:
print (data.dtypes)

Order No                             object
External Order No                     int64
Order Date                   datetime64[ns]
Order Type                           object
Status                               object
Customer Name                        object
Country                              object
State                                object
City                                 object
Email                                object
Color                                object
Quantity                            float64
Return Qty                          float64
Order Currency                       object
Price                                object
Ship Cost                           float64
Packing Cost                        float64
Discount                             object
Discount Code                        object
Tax                                  object
Invoiced                             object
Base Currency                        object
COGS                            

In [ ]:
data['Order No']. value_counts()

M0117934    183
M013430      21
M01823       20
M0113089     19
M0111025     17
           ... 
M0117019      1
M02709        1
M02712        1
M02715        1
M011000       1
Name: Order No, Length: 17641, dtype: int64

**Drop columns that are not needed**

For our Lifetime value calculation, we don't need all the features in this data set. We need only 
Customer_ID
Order No
Order Date
Quantity
MRP
Total Sales (Quantity * Price)
So, we keep only these features and drop all the others

In [ ]:
features = ['Customer_ID', 'Order No', 'Order Date', 'Quantity', 'MRP']
data_cltv = data[features]
data_cltv['TotalSales'] = data_cltv['Quantity'].multiply(data_cltv['MRP'])
print(data_cltv.shape)
data_cltv.head()

(37453, 6)


,Customer_ID,Order No,Order Date,Quantity,MRP,TotalSales
0,14955,M011000,2018-01-05 02:08:00,1.0,945,945.0
1,2532,M0110001,2019-08-22 15:12:00,1.0,3095,3095.0
2,10850,M0110002,2019-08-22 16:01:00,1.0,1095,1095.0
3,19829,M011001,2018-01-05 05:16:00,1.0,995,995.0
4,10993,M0110013,2019-08-22 22:49:00,1.0,2995,2995.0


Descrptive Statistics of the data

In [ ]:
data_cltv.describe()

,Customer_ID,Quantity,MRP,TotalSales
count,37453.000000,37453.000000,37453.000000,37453.000000
mean,10284.408592,1.039890,1080.451099,1115.502496
std,5961.739764,0.262079,586.891029,636.168929
min,1.000000,1.000000,95.000000,95.000000
25%,5072.000000,1.000000,595.000000,645.000000
50%,10239.000000,1.000000,995.000000,995.000000
75%,15491.000000,1.000000,1295.000000,1345.000000
max,20611.000000,20.000000,3495.000000,12380.000000


***Missing Values Analysis***

In [ ]:
pd.DataFrame(zip(data_cltv.isnull().sum(), data_cltv.isnull().sum()/len(data_cltv)), columns=['Count', 'Proportion'], index=data_cltv.columns)

,Count,Proportion
Customer_ID,0,0.0
Order No,0,0.0
Order Date,0,0.0
Quantity,0,0.0
MRP,0,0.0
TotalSales,0,0.0


In [ ]:
maxdate = data_cltv['Order Date'].dt.date.max() 
mindate = data_cltv['Order Date'].dt.date.min()
unique_cust = data_cltv['Customer_ID'].nunique()
tot_quantity = data_cltv['Quantity'].sum()
tot_sales = data_cltv['TotalSales'].sum()

print(f"The Time range of transactions is: {mindate} to {maxdate}")
print(f"Total number of unique customers: {unique_cust}")
print(f"Total Quantity Sold: {tot_quantity}")
print(f"Total Sales for the period: {tot_sales}")

The Time range of transactions is: 2018-01-03 to 2020-12-02
Total number of unique customers: 17202
Total Quantity Sold: 38947.0
Total Sales for the period: 41778915.0


There are no missing values in the data, hence we can proceed further with our customer life time value calculation as the data is clean without any missing values. Lets take a look at the final data which we will use in the project